In [2]:
import requests
import json
import pandas as pd
import time
import random

In [4]:
def QueryEdit(url,keyword,Affiliation=None,daterange=None,start = 0,sort = 'citedby-count'):
    url +='?query={}'.format(keyword)
    if Affiliation!=None:
        url+="+AFFIL({})".format(Affiliation)
    url += "&count={}".format(count)
    url += "&sort={}".format(sort)
    url += "&start={}".format(start)
    return url

In [10]:
conf = open('config.json','r') 
config = json.load(conf)
api_key = config['API_key']
conf.close()
keyw = open('keywords.txt','r')
keywords = keyw.readlines()
keyw.close()
for index in range(len(keywords)):
    keywords[index] = keywords[index].replace('\n','')
print(keywords)
Affiliation = 'University of Auckland'
start = 0
count = 25
step = 25
url = "https://api.elsevier.com/content/search/scopus"
headers=  {'X-ELS-APIKey':api_key}
print(url)


['clean tech', 'clean technology', 'clean-tech', 'clean energy', 'clean-energy', 'carbon emissions', 'energy efficient', 'energy-efficient', 'green energy', 'renewable energy', 'smart home', 'energy efficiency', 'geothermal', 'wastewater treatment', 'carbon emissions']
https://api.elsevier.com/content/search/scopus


In [19]:
def TidyJsontoDf(web_json,paper_info_short,column):
    try :
        for paper in web_json['search-results']['entry']:
            try:
                title = paper['dc:title']
                autor = paper['dc:creator']
                doi = paper['prism:doi']
                citedby_count = paper['citedby-count']
                affilname = paper['affiliation'][0]['affilname']
                affiliation_city = paper['affiliation'][0]['affiliation-city']
                affiliation_country = paper['affiliation'][0]['affiliation-country']
            except:
                continue
            paper_data = [[title,autor,doi,citedby_count,affilname,affiliation_city,affiliation_country]]
            paper_info = pd.DataFrame(data=paper_data, columns=column)
            paper_info_short = pd.concat([paper_info_short,paper_info],ignore_index=True)
    except:
        pass
    return paper_info_short

In [24]:
def GetAndTidy(keyword,Affiliation = None,url=url, headers = headers):
    url = QueryEdit(url,keyword,Affiliation = None)
    print(url)
    web = requests.get(url,headers=headers)
    result = web.text
    web_json = json.loads(result)
    epos = int(web_json['search-results']['opensearch:totalResults']) // step +1
    column = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
    paper_info_short = pd.DataFrame(columns=column)
    paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
    for i in range(1,epos+1):
        start = i*25
        url = "https://api.elsevier.com/content/search/scopus"
        url = QueryEdit(url,keyword,start = start)
        print(url)
        web = requests.get(url,headers=headers)
        if(web.ok):
            result = web.text
            web_json = json.loads(result)
            paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
        time.sleep(random.random())
    return paper_info_short



In [25]:
url = "https://api.elsevier.com/content/search/scopus"
paper_info = GetAndTidy(keywords[0],url = url)


https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=0
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=25
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=50
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=75
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=100
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=125
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=150
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=175
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=200
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start

In [29]:
paper_info.head()


2776

In [30]:
dois = set(paper_info.doi)
len(dois)

2776

In [178]:
web = requests.get(url,headers=headers)
result = web.text
web.ok


True

In [130]:
result

'{"service-error":{"status":{"statusCode":"INVALID_INPUT","statusText":"Exceeds the maximum number allowed for the service level"}}}'

In [236]:
web_json = json.loads(result)
print(web_json['search-results']['opensearch:totalResults'])
epos = int(web_json['search-results']['opensearch:totalResults']) // 25 +1


3219


In [238]:
for i in range(1,epos+1):
    start = i*25
    url = "https://api.elsevier.com/content/search/scopus"
    url = QueryEdit(url,keyword,start = start)
    print(url)
    web = requests.get(url,headers=headers)
    print(web.ok)
    result = web.text
    colnum = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
    paper_info_short = pd.DataFrame(columns=colnum)
    



https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=25
True
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=50
True
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=75
True
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=100
True
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=125
True
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=150
True
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=175
True
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=200
True
https://api.elsevier.com/content/search/scopus?query=cleantech&count=25&sort=citedby-count&start=225
True
https://api.elsevier.com/content/search/scopus?qu

In [180]:
f = open('scopus_raw_result.json','w+')
json_object = json.dumps(web_json, indent=4)
f.write(json_object)
f.close()


In [220]:
colnum = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
paper_info_short = pd.DataFrame(columns=colnum)

In [222]:
paper_info_short = paper_info_short.reset_index(drop=True)
   

In [225]:
101 // 25

4

In [81]:
web_json['search-results']['entry'][0]['affiliation']

[{'@_fa': 'true',
  'affilname': 'School of Civil and Environmental Engineering',
  'affiliation-city': 'Singapore City',
  'affiliation-country': 'Singapore'},
 {'@_fa': 'true',
  'affilname': 'Shenzhen University',
  'affiliation-city': 'Shenzhen',
  'affiliation-country': 'China'}]

In [172]:
aburl = 'https://api.elsevier.com/content/article/doi/10.1016/j.envpol.2022.120034'
abstract = requests.get(aburl,headers = headers)

In [174]:
print(abstract.text)
temp = open("temp.txt",'w+')
temp.write(abstract.text)
temp.close()

<full-text-retrieval-response xmlns="http://www.elsevier.com/xml/svapi/article/dtd" xmlns:bk="http://www.elsevier.com/xml/bk/dtd" xmlns:cals="http://www.elsevier.com/xml/common/cals/dtd" xmlns:ce="http://www.elsevier.com/xml/common/dtd" xmlns:ja="http://www.elsevier.com/xml/ja/dtd" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:sa="http://www.elsevier.com/xml/common/struct-aff/dtd" xmlns:sb="http://www.elsevier.com/xml/common/struct-bib/dtd" xmlns:tb="http://www.elsevier.com/xml/common/table/dtd" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xocs="http://www.elsevier.com/xml/xocs/dtd" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:prism="http://prismstandard.org/namespaces/basic/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><coredata><prism:url>https://api.elsevier.com/content/article/pii/S0269749122012489</prism:url><dc:identifier>doi:10.1016/j.envpol.2022.120034</dc:identifier><eid>1-s2.0-S0269749122012489</eid><pris